##### *Copyright 2020 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retrain a classification model for AIY Vision Kit (with TF1)

In this tutorial, we'll use TensorFlow to retrain an image classification model (MobileNet) with a flowers dataset, and convert it into the TensorFlow Lite format that's compatible with the AIY Vision Bonnet (included in the AIY Projects Vision Kit).


**Note:** The scripts used in this tutorial depend on TensorFlow 1.7.

<a href="https://colab.research.google.com/github/scottamain/aiyprojects-raspbian/blob/colab/tutorials/vision/aiy_tf_for_poets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/scottamain/aiyprojects-raspbian/blob/colab/tutorials/vision/aiy_tf_for_poets.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


To start running all the code in this tutorial, select **Runtime > Run all** in the Colab toolbar.

## Import the required libraries

First, we need to uninstall the version of TensorFlow that's included with Google Colab by default, and replace it with TensorFlow 1.7, as required by the following training scripts.

In [ ]:
! pip uninstall tensorflow tensorboard -y

In [ ]:
! pip install -I absl-py==0.9 jupyter-client==6.1.5 tornado==5.1.0 folium==0.2.1 imgaug==0.2.5 tensorflow==1.7 tensorboard==1.7

In [ ]:
import tensorflow as tf
assert tf.__version__.startswith('1.7')

## Prepare the training data

First let's download and organize the flowers dataset we'll use to retrain the model (it contains 5 flower classes).

Pay attention to this part so you can reproduce it with your own images dataset. In particular, notice that the "flower_photos" directory contains an appropriately-named directory for each class.

In [ ]:
! git clone https://github.com/googlecodelabs/tensorflow-for-poets-2

%cd tensorflow-for-poets-2

In [ ]:
! curl http://download.tensorflow.org/example_images/flower_photos.tgz | tar xz -C tf_files

In [ ]:
! pwd

In [ ]:
! ls tf_files/flower_photos

## Retrain the model

First specify the input image size (this is for both width and height; the model's input expects a square image) and depth multiplier for the MobileNet model. 

Based on our testing, the only variations compatible with the Vision Bonnet's ML accelerator are the following:

+ Input size = 160x160, and depth multiplier = 0.5
+ Input size = 192x192, and depth multiplier = 1.0

In [ ]:
IMAGE_SIZE='160'
MULTIPLIER='0.50'
%env ARCHITECTURE=mobilenet_{MULTIPLIER}_{IMAGE_SIZE}

Then start training:


In [ ]:
! python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/$ARCHITECTURE \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture=$ARCHITECTURE \
  --image_dir=tf_files/flower_photos

## Compile the model for the Vision Kit

The training script above creates a TensorFlow Lite model that you can run on a CPU, but we want to run this on the AIY Vision Bonnet's ML accelerator (the Myriad 2450). So we need to compile the model for that chip.

First download the Vision Bonnet model compiler:

In [ ]:
! curl -LO https://dl.google.com/dl/aiyprojects/vision/bonnet_model_compiler_latest.tgz	

! tar -xzf bonnet_model_compiler_latest.tgz

Then compile the model:

In [ ]:
! ./bonnet_model_compiler.par \
  --frozen_graph_path=tf_files/retrained_graph.pb \
  --output_graph_path=tf_files/retrained_graph.binaryproto \
  --input_tensor_name=input \
  --output_tensor_names=final_result \
  --input_tensor_size=160 \
  --debug

Don't worry if you see an error like this:
`Check failed: toco::port::file::GetContents(FLAGS_frozen_graph_path, &frozen_graph_contents, toco::port::file::Options()) .ok()`
That can happen if the compiler tried requesting the model file before Colab was done saving it from the previous step. So just click the Play button in the above code to run it again. It should work this time.

That's it. Your retrained model is ready to run on the Vision Kit.

## Download the model


The compiled model is saved into this Colab runtime's temporary storage. You can download it as follows:

1. Open the **Files** tab in the left panel.
2. Expand the **tensorflow-for-poets-2** folder and then the **tf_files** folder.
3. Right-click on `retrained_graph.binaryproto` and select **Download**.
4. Also download `retrained_labels.txt`.

## Run the model on the Vision Kit




You can now run the model on your Coral device with acceleration on the Edge TPU.

To get started, try using [this code for image classification with the TensorFlow Lite API](https://github.com/google-coral/tflite/tree/master/python/examples/classification). Just follow the instructions on that page to set up your device, copy the `mobilenet_v2_1.0_224_quant_edgetpu.tflite` and `flower_labels.txt` files to your Coral Dev Board or device with a Coral Accelerator, and pass it a flower photo like this:

```
python3 classify_image.py \
  --model mobilenet_v2_1.0_224_quant_edgetpu.tflite \
  --labels flower_labels.txt \
  --input flower.jpg
```

Check out more examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).